In [1]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import pandas as pd
import eipy.ei as e
from eipy.additional_ensembles import MeanAggregation, CES

In [2]:
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

In [3]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True),
}

In [4]:
import pickle as pkl
with open("/home/opc/eipy/tadpole/tadpole_data_time_imptn_norm_v1.pickle", "rb") as file:
    data = pkl.load(file)
with open("/home/opc/eipy/tadpole/tadpole_labels_time_imptn_norm_v1.pickle", "rb") as file:
    labels = pkl.load(file)

In [5]:
data = [data[k] for k in data.keys()]

In [6]:
for k,v in labels.items():
    labels[k] = v.reset_index(drop=True)

In [7]:
#intermediate transformation to make sure labels are ordered correctly in time
labels = pd.DataFrame(labels)

labels = labels.to_numpy()

In [8]:
import numpy as np
encoding_dict = {'NL': 0, 'MCI': 1, 'Dementia': 2}

# Use numpy.vectorize with a lambda function to apply the encoding
labels = np.vectorize(lambda x: encoding_dict[x])(labels)

In [9]:
labels.shape

(702, 5)

In [10]:
for i in range(5):
    print(pd.Series(labels[:,i]).value_counts())

1    502
0    200
Name: count, dtype: int64
1    480
0    195
2     27
Name: count, dtype: int64
1    451
0    192
2     59
Name: count, dtype: int64
1    378
0    188
2    136
Name: count, dtype: int64
1    348
2    178
0    176
Name: count, dtype: int64


In [11]:
dem_freq = np.sum(labels == 2, axis=1)

In [12]:
pd.Series(dem_freq).value_counts()

0    378
1    250
2     72
3      2
Name: count, dtype: int64

In [13]:
labels

array([[0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1],
       [0, 0, 0, 0, 0],
       ...,
       [0, 0, 1, 1, 1],
       [0, 1, 1, 1, 1],
       [0, 1, 1, 1, 1]])

In [14]:
####HI OVER HERE###
import importlib
importlib.reload(e)
###HEYOOOOOO#

<module 'eipy.ei' from '/home/opc/eipy/eipy/ei.py'>

In [15]:
outer_folds = 5

In [16]:
meta_data = []
for t in range(len(data)):
    #time dependent data splitting
    X_train_timestep = data[t]
    labels_at_timestep = labels[:, t]
    EI_for_timestep = e.EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy="oversampling",
                        sampling_aggregation="mean",
                        n_jobs=-1,
                        metrics=metrics,
                        random_state=38,
                        project_name=f"time step {t}",
                        model_building=False,
                        )
    print(f"generating metadata for timestep {t}")
    EI_for_timestep.final_label_frequency = dem_freq
    EI_for_timestep.fit_base(X_train_timestep, labels_at_timestep)
    meta_data.append([EI_for_timestep.ensemble_training_data, EI_for_timestep.ensemble_test_data, EI_for_timestep.ensemble_training_data_final, EI_for_timestep.base_summary])


generating metadata for timestep 0
Training base predictors on Main cognitive tests...
        
... for ensemble performance analysis...


Generating ensemble training data: |          |  0%/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
Generating ensemble training data: |██████████|100%
Generating ensemble test data: |          |  0%/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_se



Training base predictors on MRI volumes...
        
... for ensemble performance analysis...


Generating ensemble training data: |          |  0%/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
Generating ensemble training data: |██████████|100%
Generating ensemble test data: |          |  0%/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_se



Training base predictors on Demo, APOE4 & others...
        
... for ensemble performance analysis...


...

Generating ensemble training data: |          |  0%/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
Generating ensemble training data: |██████████|100%
Generating ensemble test data: |          |  0%/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_se



Training base predictors on MRI ROI: Cortical Thickness Average...
        
... for ensemble performance analysis...


Generating ensemble training data: |          |  0%/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
Generating ensemble training data: |██████████|100%
Generating ensemble test data: |          |  0%/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_se



Training base predictors on MRI ROI: Cortical Thickness Standard Deviation...
        
... for ensemble performance analysis...


Generating ensemble training data: |          |  0%/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
Generating ensemble training data: |██████████|100%
Generating ensemble test data: |          |  0%/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_se

In [17]:
RNN_training_data = [[dfs[0][i] for dfs in meta_data] for i in range(5)]
RNN_test_data = [[dfs[1][i] for dfs in meta_data] for i in range(5)]
RNN_training_data_final = [df[2] for df in meta_data]

### make first time point in meta-data multiclass

In [18]:
def get_column_names(frame):
    column_names = []
    for i in range(frame.columns.nlevels):
        if i == 0:
            column_names.append(frame.columns.get_level_values(i).unique().drop("labels"))
            
        else:
            column_names.append(frame.columns.get_level_values(i).unique().drop(''))
    
    return column_names

In [19]:
def fix_first_time_point(df):
    new_columns = get_column_names(df)
    classes=[0,1,2]
    new_columns.append(classes)
    new_mux=pd.MultiIndex.from_product(iterables=new_columns, names=["modality", "base predictor", "sample", "class"])
    new_df = pd.DataFrame(columns=new_mux)

    for col in new_df.columns:
        if col[-1] == 0:
            new_df[col] = 1 - df[col[:-1]]
        elif col[-1] == 1:
            new_df[col] = df[col[:-1]]
        else:
            new_df[col] = 0
    
    new_df['labels'] = df['labels']

    return new_df

In [20]:
for i in range(len(RNN_training_data)):
    RNN_training_data[i][0] = fix_first_time_point(RNN_training_data[i][0])
    RNN_test_data[i][0] = fix_first_time_point(RNN_test_data[i][0])

/tmp/ipykernel_2577649/2424389332.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df['labels'] = df['labels']
/tmp/ipykernel_2577649/2424389332.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df['labels'] = df['labels']
/tmp/ipykernel_2577649/2424389332.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `new

# TIME SERIES TIME

In [21]:
from keras import backend as K

def ordinal_regression_loss(y_true, y_pred):
    """
    Ordinal regression loss function.

    Parameters:
    - y_true: true labels (ground truth)
    - y_pred: predicted labels

    Returns:
    - loss value
    """
    # Assuming y_true and y_pred are tensors of shape (batch_size, num_classes)

    # Calculate cumulative probabilities for true and predicted labels
    true_cum_probs = K.cumsum(K.softmax(y_true, axis=-1), axis=-1)
    pred_cum_probs = K.cumsum(K.softmax(y_pred, axis=-1), axis=-1)

    # Calculate the ordinal regression loss
    loss = K.sum((true_cum_probs - pred_cum_probs) ** 2)

    return loss

2024-01-08 21:32:34.165153: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-08 21:32:34.193103: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 21:32:34.193128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 21:32:34.193944: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 21:32:34.198535: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-08 21:32:34.199180: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [22]:
from keras.models import Sequential
from keras.layers import LSTM,Dense
import tensorflow as tf
y_preds = [] # LSTM predictions at every time point. Will be populated by 5 arrays
for i in range(len(RNN_training_data)):
    tf.random.set_seed(38)
    lstm = Sequential()
    lstm.add(LSTM(units=50, input_shape=(5,240), return_sequences=True))
    lstm.add(Dense(units=3, activation='softmax'))
    lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) #loss='categorical_crossentropy'


    labels_across_time = np.column_stack([df['labels'].values for df in RNN_training_data[i]])
    labels_across_time = np.eye(3)[labels_across_time]

    RNN_training_data_fold = [data.drop(columns=["labels"], axis=1) for data in RNN_training_data[i]]
    data_arrays_per_timepoint = [df.to_numpy() for df in RNN_training_data_fold]
    tensor_3d = np.stack(data_arrays_per_timepoint, axis=0)
    tensor_3d = np.transpose(tensor_3d, (1,0,2))

    lstm.fit(tensor_3d, labels_across_time)

    RNN_test_data_fold = [data.drop(columns=["labels"], axis=1) for data in RNN_test_data[i]]
    data_arrays_per_timepoint_test = [df.to_numpy() for df in RNN_test_data_fold]
    tensor_3d_test = np.stack(data_arrays_per_timepoint_test, axis=0)
    tensor_3d_test = np.transpose(tensor_3d_test, (1,0,2))

    y_preds.append(lstm.predict(tensor_3d_test))

/tmp/ipykernel_2577649/2502916098.py:16: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  RNN_training_data_fold = [data.drop(columns=["labels"], axis=1) for data in RNN_training_data[i]]


18/18 [==============================] - 1s 3ms/step - loss: 0.7955 - accuracy: 0.6656


/tmp/ipykernel_2577649/2502916098.py:23: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  RNN_test_data_fold = [data.drop(columns=["labels"], axis=1) for data in RNN_test_data[i]]


5/5 [==============================] - 0s 2ms/step


In [23]:
y_preds_argmax = [np.argmax(pred, axis=-1) for pred in y_preds]

In [24]:
y_trues = []
for i in range(len(RNN_test_data)):
    y_true = pd.concat([data["labels"] for data in RNN_test_data[i]], axis=1).to_numpy()
    y_trues.append(y_true)


In [25]:
from sklearn.metrics import roc_auc_score
aucs=[]
for i in range(len(y_preds)):
    aucs_for_fold=[]
    for j in range(1, y_preds[i].shape[-2]):
        aucs_for_fold.append(roc_auc_score(y_trues[i][:,j], y_preds[i][:,j], multi_class='ovr'))
    aucs.append(aucs_for_fold)

auc_df = pd.DataFrame(data=aucs, columns=["m06", "m12", "m24", "m36"])
auc_df


,m06,m12,m24,m36
0,0.936238,0.889276,0.874553,0.894433
1,0.935118,0.890685,0.904047,0.879086
2,0.938372,0.887215,0.896461,0.881070
3,0.924087,0.885767,0.868845,0.890868
4,0.909504,0.910450,0.873962,0.887788


In [26]:
from sklearn.metrics import f1_score
f1s=[]
for i in range(len(y_preds_argmax)):
    f1s_for_fold=[]
    for j in range(y_preds_argmax[i].shape[-1]):
        f1s_for_fold.append(f1_score(y_preds_argmax[i][:,j], y_trues[i][:,j], average='micro'))
    f1s.append(f1s_for_fold)

f1_df = pd.DataFrame(data=f1s, columns=["bl", "m06", "m12", "m24", "m36"])
f1_df


,bl,m06,m12,m24,m36
0,0.950355,0.851064,0.765957,0.659574,0.687943
1,0.929078,0.872340,0.794326,0.730496,0.673759
2,0.928571,0.807143,0.764286,0.542857,0.735714
3,0.942857,0.864286,0.785714,0.685714,0.685714
4,0.907143,0.864286,0.828571,0.657143,0.600000


In [27]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

confusion_matrices = [ [confusion_matrix(y_pred=y_preds_argmax[i][:,j], y_true=y_trues[i][:,j]) for j in range(5)] for i in range(len(y_trues))]

In [28]:
final_f1_dem = []
for i in range(len(y_preds_argmax)):
    print(f" \n FOLD {i+1} \n")
    for j in range(5):
        print(classification_report(y_pred=y_preds_argmax[i][:,j], y_true=y_trues[i][:,j]))

 
 FOLD 1 

              precision    recall  f1-score   support

           0       0.97      0.85      0.90        39
           1       0.94      0.99      0.97       102

    accuracy                           0.95       141
   macro avg       0.96      0.92      0.94       141
weighted avg       0.95      0.95      0.95       141

              precision    recall  f1-score   support

           0       0.87      0.71      0.78        38
           1       0.85      0.96      0.90        97
           2       0.00      0.00      0.00         6

    accuracy                           0.85       141
   macro avg       0.57      0.56      0.56       141
weighted avg       0.82      0.85      0.83       141

              precision    recall  f1-score   support

           0       0.71      0.59      0.65        34
           1       0.78      0.92      0.84        96
           2       0.00      0.00      0.00        11

    accuracy                           0.77       141
   macro

In [29]:
import re
target_class=2
pattern = re.compile(rf"\b{target_class}\b\s+\d+\s+\d+\s+([\d.]+)\s+")
match = re.search(pattern, classification_report(y_pred=y_preds_argmax[0][:,-1], y_true=y_trues[0][:,-1]))

if match:
    f1_score_for_target_class = float(match.group(1))
    print(f"F1 Score for class {target_class}: {f1_score_for_target_class}")
else:
    print(f"No F1 Score found for class {target_class}")

No F1 Score found for class 2
